In [2]:
import open3d as o3d
import numpy as np
import copy
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
mesh_path = './models/ARIC_board_3.stl'
mesh = o3d.io.read_triangle_mesh(mesh_path)
#mesh.compute_vertex_normals()
mesh.compute_triangle_normals()

print(mesh.is_watertight()) #great. goign to reconstruct surface

point_cloud = mesh.sample_points_uniformly(1500)
point_cloud.estimate_normals()
depth = 9
mesh,densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(point_cloud, depth=depth)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])
# all black but it is watertight, should be able to subtrat now


: 

: 

In [ ]:
box = mesh.get_minimal_oriented_bounding_box(robust=True)

box_mesh = mesh.create_box(box.extent[0], box.extent[1], box.extent[2])

mesh = mesh.translate((-1)*mesh.get_center())

# Step 2: rotating to align with x axis: assuming R is relative to center
R_inv = np.linalg.inv(box.R)
mesh.rotate(R_inv, center=(0, 0, 0))

# new test for Step 3
mesh = mesh.translate(box.extent*(0.5))

mesh_coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=50, origin=[0, 0, 0])
box_mesh.compute_triangle_normals()
box_mesh_c = copy.deepcopy(box_mesh).scale(0.90, center=box_mesh.get_center())
# Save Images
#o3d.io.write_triangle_mesh("models/aligned_defect_mesh.stl", mesh)
#o3d.io.write_triangle_mesh("models/aligned_rect_mesh.stl", box_mesh)

o3d.visualization.draw_geometries([mesh, box_mesh_c])



In [4]:
# https://stackoverflow.com/questions/61370374/boolean-geometry-subtraction-of-3d-meshes

import pymeshlab

ms = pymeshlab.MeshSet()
ms.load_new_mesh("models/aligned_defect_mesh.stl")

vol = ms.get_geometric_measures()
print(vol)
ms.load_new_mesh("models/aligned_rect_mesh.stl")
ms.generate_boolean_intersection()
ms.save_current_mesh("models/pymeshlab_intersection.stl")

{'avg_edge_inc_faux_length': 3.8860703823131906, 'avg_edge_length': 3.8860703823131906, 'axis_momenta': array([1.15760485e+09, 3.12305984e+10, 3.20206168e+10]), 'barycenter': array([245.69650467,  47.5577018 ,  22.8488418 ]), 'bbox': <pymeshlab.pmeshlab.BoundingBox object at 0x7f9ccc7a0870>, 'center_of_mass': array([245.02961025,  48.20567154,  22.90605161]), 'inertia_tensor': array([[ 1.15780666e+09,  2.82500481e+07,  7.35198627e+07],
       [ 2.82500481e+07,  3.12316808e+10, -2.96467996e+07],
       [ 7.35198627e+07, -2.96467996e+07,  3.20193326e+10]]), 'mesh_volume': 1583582.7603906174, 'pca': array([[-9.99996717e-01,  9.41695023e-04,  2.38313098e-03],
       [ 1.03032165e-03,  9.99297395e-01,  3.74653720e-02],
       [-2.34617563e-03,  3.74677044e-02, -9.99295085e-01]]), 'shell_barycenter': array([244.20551601,  48.06927428,  23.09029472]), 'surface_area': 120109.046875, 'total_edge_inc_faux_length': 122527.7991543349, 'total_edge_length': 122527.7991543349}


In [5]:
diff = o3d.io.read_triangle_mesh("models/pymeshlab_intersection.stl")
diff.compute_triangle_normals()
o3d.visualization.draw_geometries([diff])
#FAIL!!

In [11]:
mesh = o3d.io.read_triangle_mesh("def_mesh_aligned.stl")
mesh.compute_triangle_normals()
box = mesh.get_minimal_oriented_bounding_box()
box.color = (1,0,0)

o3d.visualization.draw_geometries([mesh])

In [10]:
diff = o3d.io.read_triangle_mesh("rect_mesh_aligned.stl")
diff.compute_triangle_normals()
o3d.visualization.draw_geometries([diff])